In [10]:
from procDataSet import TrainingQuery
from bert_serving.client import BertClient
from multiprocessing import Pool
from torch.utils.data import TensorDataset, DataLoader, Dataset
import torch
import torch.nn as nn
import torch.optim  as optim
import argparse
from keras.utils import to_categorical
import pandas as pd
import numpy as np
from torch.functional import F
import os
import jieba
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import clear_output
import jieba.posseg as pseg
jieba.load_userdict(os.path.join('data', 'dict.txt.big'))
[jieba.add_word(i, freq=None, tag=None) for i in ['不支持','文林苑', '都更案','十八趴','證所稅','前瞻建設']]

[None, None, None, None, None, None]

In [11]:
top_num = 300

In [12]:
with open('data/stop_word_zhtw.txt') as file:
    data = file.read()
    
stop_words = data.split('\n')
stop_words += ['「', '」', '，', '\n', '）', '（', ')', '(']

In [13]:
from sklearn.model_selection import train_test_split
folder = 'data/'
raw_training_data = pd.read_csv(os.path.join(folder,'TD.csv'))
news_urls = pd.read_csv(os.path.join(folder,'NC_1.csv'))
contents = pd.read_json(os.path.join(folder,'url2content.json'), typ=pd.Series)

## sort the contents by index
keys, content_list = contents.keys(), contents.values

In [14]:
test_query = np.array(pd.read_csv('./data/QS_1.csv').Query)

In [15]:
for test_id , text_q in enumerate(test_query):
    print(jieba.lcut_for_search(text_q))

['通姦', '在', '刑法', '上', '應該', '除罪', '罪化', '除罪化']
['應該', '取消', '機車', '強制', '二段', '段式', '二段式', '左轉', '(', '待轉', ')']
['支持', '博弈', '特區', '在', '台灣', '合法', '合法化']
['中華', '航空', '空服', '服員', '空服員', '罷工', '是', '合理', '的']
['性交', '交易', '性交易', '應該', '合法', '合法化']
['ECFA', '早', '收清', '清單', '收清單', '可', '（', '有', '）', '達到', '其', '預期', '成效']
['應該', '減免', '所稅', '證所稅']
['贊成', '中油', '在', '觀塘', '興建', '第三', '天然', '天然氣', '接收', '接收站']
['支持', '中國', '學生', '納入', '健保']
['支持', '臺灣', '中小', '小學', '中小學', '（', '含', '高職', '、', '專科', '）', '服儀', '規定', '（', '含髮', '、', '襪', '、', '鞋', '）', '給予', '學生', '自主']
['不支', '支持', '不支持', '使用', '加密', '貨幣']
['不支', '支持', '不支持', '雜費', '學雜費', '調漲']
['同意', '政府', '舉債', '發展', '前瞻', '建設', '前瞻建設', '計畫']
['支持', '電競', '列入', '體育', '競技', '體育競技']
['反對', '台鐵', '東移', '徵收', '收案', '徵收案']
['支持', '陳前', '總統', '前總統', '陳前總統', '保外', '就醫', '保外就醫']
['年金', '改革', '應', '取消', '或', '應', '調降', '公教', '軍公教', '月', '退之', '優存', '利率', '十八', '十八趴']
['同意', '動物', '實驗']
['油價', '應該', '凍漲', '或', '緩漲']
['反對', '旺旺', '中時', '併購', '中嘉

In [ ]:
import multiprocessing

NumberCPU = multiprocessing.cpu_count()
jieba.initialize()

def jbcut(x):
    if x is not None:
        sen = jieba.lcut(x, cut_all=False)
        sen = [i for i in sen if i not in stop_words]
        return sen
    else:
        return None

def psegcut(x):
    if x is not None:
        sen = pseg.lcut(x)
        sen = [i for i in sen if i not in stop_words]
        return sen
    else:
        return None
    
pool = multiprocessing.Pool(processes=NumberCPU)
sentenece_arr = pool.map(jbcut,content_list)
pool.close()
pool.join()

In [16]:
sentence_arr = np.load('arr.npy', allow_pickle=True)

## okapi model

In [17]:
from gensim.summarization import bm25
bm25Model = bm25.BM25(sentence_arr)

In [18]:
stands = ['支持', '應該', '反對', '贊成', '不'] 

In [19]:
train_query = list(set(raw_training_data.Query))

y_train = []
y_index = {}
for i  in train_query:
    index = np.where(raw_training_data.Query==i)
    data = raw_training_data.iloc[index]
    y = dict(zip(data.News_Index,data.Relevance))
    y_idx = [ (int(idx.split('_')[1])-1, rel )  for idx, rel in zip(data.News_Index, data.Relevance)]
    y_train.append(y)
    y_index[i] = [y_idx]

In [20]:
common_query = set(train_query) & set(test_query)
common_query = list(common_query)
common = [ train_query.index(i) for i in common_query]

In [21]:
[ len(list(y_train[i].values())) for i in common]

[95, 230, 40, 139, 230]

In [22]:
[ np.sum(list(y_train[i].values())) / 300 for i in range(20)]

[0.23,
 2.7866666666666666,
 0.5766666666666667,
 2.1933333333333334,
 1.3233333333333333,
 0.2833333333333333,
 0.9433333333333334,
 0.7066666666666667,
 0.77,
 1.0533333333333332,
 0.14,
 2.296666666666667,
 0.6933333333333334,
 0.21333333333333335,
 1.35,
 1.4466666666666668,
 0.24333333333333335,
 1.3,
 2.6466666666666665,
 1.3733333333333333]

In [ ]:
# [ np.sum(list(y_train[i].values())) / 300 for i in common]

In [23]:
total_scores = list()
for test_id , text_q in enumerate(train_query):
    text = [ str(i) for i in pseg.lcut(text_q)]
    text = [ t for t in text if t not in stop_words]
    scores = np.array(bm25Model.get_scores(text))
    top_query = np.argsort(scores)[::-1][:80]
    irrelevant_query = np.argsort(scores)[:80]
    
    print(test_id ,text_q, text)
    print("top query num {}".format(len(top_query)))
    
    
    for query in top_query:
        all_words = [ (sentence_arr[query][cnt], bm25Model.get_score(sentence_arr[query], cnt))
                     for cnt, i in enumerate(sentenece_arr[query])]

        all_words = sorted(all_words,key=lambda x:(x[1]))[::-1]
        top_words = all_words[:20]
        top_words = [x[0] for x in top_words]
        text += top_words
    scores += 0.3 * np.array(bm25Model.get_scores(text))
    
    irrelevant = []
    for query in irrelevant_query:
        all_words = [ (sentence_arr[query][cnt], bm25Model.get_score(sentence_arr[query], cnt))
                     for cnt, i in enumerate(sentenece_arr[query])]

        all_words = sorted(all_words,key=lambda x:(x[1]))[::-1]
        top_words = all_words[:20]
        top_words = [x[0] for x in top_words]
        irrelevant += top_words
    
    scores -= 0.3 * np.array(bm25Model.get_scores(irrelevant))
#     for idx, rel in (y_index[text_q][0]):
#         scores[idx] += np.mean(scores) * idx
    
    top_num = 300
    keys = pd.DataFrame(np.argsort(scores)[::-1][:top_num])
    ans = keys[0].apply(lambda x: 'news_{:06d}'.format(x+1))
    
    validation_score = 0
    for a in ans:
        if a in y_train[test_id].keys():
            validation_score += y_train[test_id][a]
            
    total_scores.append(validation_score/top_num)
    print("{} / {} ".format(validation_score,(np.sum(list(y_train[test_id].values())))))

0 油價應該凍漲或緩漲 ['油價/n', '應該/v', '凍漲/Vi', '或/c', '緩漲/Vi']
top query num 80


NameError: name 'sentenece_arr' is not defined

$${\displaystyle k_{1}\in [1.2,2.0]}, {\displaystyle b=0.75}$$

In [ ]:
total_scores = list()
for test_id , text_q in enumerate(test_query):
    text = jieba.lcut(text_q)
    text = [ t for t in text if t not in stop_words]
    scores = bm25Model.get_scores(text)
    top_query = np.argsort(scores)[::-1][:100]
    irrelevant_query = np.argsort(scores)[:200]
    
    print(test_id ,text_q, text)
    print("top query num {}".format(len(top_query)))
    
    for query in top_query:
        all_words = [ (sentence_arr[query][cnt], bm25Model.get_score(sentence_arr[query], cnt))
                     for cnt, i in enumerate(sentenece_arr[query])]

        all_words = sorted(all_words,key=lambda x:(x[1]))[::-1]
        top_words = all_words[:20]
        top_words = [x[0] for x in top_words]
        text += top_words
    scores += 0.3 * np.array(bm25Model.get_scores(text))
    
    irrelevant = []
    for query in irrelevant_query:
        all_words = [ (sentence_arr[query][cnt], bm25Model.get_score(sentence_arr[query], cnt))
                     for cnt, i in enumerate(sentenece_arr[query])]

        all_words = sorted(all_words,key=lambda x:(x[1]))[::-1]
        top_words = all_words[:20]
        top_words = [x[0] for x in top_words]
        irrelevant += top_words
    
    scores -= 0.3 * np.array(bm25Model.get_scores(irrelevant))
    
#     scores = bm25Model.get_scores(text)
    delta = np.max(scores) 
    if text_q in y_index.keys():
        for idx, rel in (y_index[text_q][0]):
            scores[idx] += delta * idx
            
    total_scores.append(scores)

## Test the model

In [ ]:
top_num = 300
search_result = np.zeros((20,top_num))
for cnt,i in enumerate(total_scores):
    keys = np.argsort(i)[::-1][:top_num]
    search_result[cnt] += keys
    
search_result = search_result.astype(np.int)

df = pd.DataFrame()
df['Query_Index'] = ['q_{:02d}'.format(i+1) for i in range(20)]

for i in range(top_num):
    df['Rank_{:03d}'.format(i+1)] = search_result[:, i] + 1

for i, row in df.iterrows():
    df.iloc[i, 1:] = df.iloc[i, 1:].apply(lambda x: 'news_{:06d}'.format(x))

In [ ]:
import os
sorted(os.listdir('output/'))

In [ ]:
fname = 'simple23.csv'
df.to_csv('output/' + fname, index=False)

# search_result[0]